In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newprediction21/NewPrediction.csv
/kaggle/input/prediction3/WhoKnowsPred.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv
/kaggle/input/prediction1/Prediction.csv


****Below is first model in which like missing values are replaced by overall median****

In [2]:
import pandas as pd 
df=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
df.fillna(df.median(numeric_only=True),inplace=True)
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Drop ID column
df.drop(columns=['ID'], inplace=True)

# Encode class column (if it's categorical)
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

# Split into features and target
X = df.drop(columns=['class'])
y = df['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Fit multinomial logistic regression
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000
)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Classification report with all original class labels
print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_
))
model.score(X_train,y_train)
model.score(X_test,y_test)
X_test
test_set=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
ids=test_set["ID"]
test_set.drop(columns=["ID"],inplace=True)
x1=test_set
y1=model.predict(x1)
y1
pred_class=label_encoder.inverse_transform(y1)
final_dataset=pd.DataFrame({"ID":ids,"class":pred_class})
final_dataset["class"].value_counts()
final_dataset.to_csv("Prediction.csv",index=False)
print(label_encoder.classes_)
df
model.score(X_train,y_train)
#model.score(X_test,y_test)

              precision    recall  f1-score   support

        farm       0.77      0.80      0.79       168
      forest       0.98      0.99      0.98      1232
       grass       0.62      0.46      0.53        39
  impervious       0.75      0.82      0.79       134
     orchard       0.50      0.17      0.25         6
       water       0.92      0.52      0.67        21

    accuracy                           0.93      1600
   macro avg       0.76      0.63      0.67      1600
weighted avg       0.93      0.93      0.93      1600

['farm' 'forest' 'grass' 'impervious' 'orchard' 'water']


0.94796875

****So like for second model what we tried to is like replace the missing values with median of respective values for each class and thats what worked better***

In [3]:
dataset=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
dataset1=dataset[pd.notnull(dataset["20150720_N"])]
class_median=pd.DataFrame(dataset1.groupby("class")[column].median() for column in dataset.columns.values[3:])
class_median.reset_index()
def datafix(row):
    columns=dataset.columns.values[3:]
    for column in columns:
      if pd.isna(row[column]) and row["class"]=="farm":
          row[column]=class_median.loc[column,"farm"]
      if pd.isna(row[column]) and row["class"]=="forest":
          row[column]=class_median.loc[column,"forest"]
      if pd.isna(row[column]) and row["class"]=="grass":
          row[column]=class_median.loc[column,"grass"]
      if pd.isna(row[column]) and row["class"]=="impervious":
          row[column]=class_median.loc[column,"impervious"]
      if pd.isna(row[column]) and row["class"]=="water":
          row[column]=class_median.loc[column,"water"]
      if pd.isna(row[column]) and row["class"]=="orchard":
          row[column]=class_median.loc[column,"orchard"]
    return row
dataset=dataset.apply(datafix,axis="columns")

***Standardization was also done along with better replacement of missing values***

In [4]:
data=dataset.copy()
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

# Drop ID column

# Encode class column (if it's categorical)
label_encoder3 = LabelEncoder()
data['class'] = label_encoder3.fit_transform(data['class'])

# Split into features and target
X = data.iloc[:,3:]
y = data['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Fit multinomial logistic regression
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=2000
)
model.fit(X_train_scaled, y_train)
# Predict on test set
y_pred = model.predict(X_test_scaled)

# Classification report with all original class labels
'''print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder3.classes_))),
    target_names=label_encoder3.classes_
))'''
model.score(X_train_scaled,y_train)
model.score(X_test_scaled,y_test)
import matplotlib.pyplot as plt
import seaborn as sns
y_test_decoded=label_encoder3.inverse_transform(y_test)
y_pred_decoded=label_encoder3.inverse_transform(y_pred)
cm=confusion_matrix(y_test_decoded,y_pred_decoded)
plt.figure(figsize=(10, 8))
'''sns.heatmap(cm, 
            annot=True,  # Show numbers in cells
            fmt='d',     # Format as integers
            cmap='Blues',
            xticklabels=label_encoder3.classes_,
            yticklabels=label_encoder3.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()'''
old_pred=pd.read_csv("/kaggle/input/prediction1/Prediction.csv")
old_pred["class"].value_counts()
new_dataset=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
ids=new_dataset["ID"]
x_new=new_dataset.iloc[:,2:]
x_new
x_scaled=scaler.transform(x_new)
x_scaled
y_new=model.predict(x_scaled)
y_pred_new=label_encoder3.inverse_transform(y_new)
y_pred_new
dfinal=pd.DataFrame({"ID":ids,"class":y_pred_new})
old_pred["class"].value_counts()
dfinal["class"].value_counts()
dfinal.to_csv("NewPrediction23.csv",index=False)

<Figure size 1000x800 with 0 Axes>

****Below is some like analysis of previous versus new predictions****

In [5]:
# Load both prediction files
old_predictions = pd.read_csv("NewPrediction23.csv")  # Previous prediction with score of 0.53136
new_predictions = pd.read_csv("/kaggle/input/prediction3/WhoKnowsPred.csv")      # New prediction

# Checking the different predictions
predictions_changed = (old_predictions['class'] != new_predictions['class']).sum()
total_predictions = len(old_predictions)

print(f"Total predictions: {total_predictions}")
print(f"Changed predictions: {predictions_changed}")
print(f"Percentage changed: {predictions_changed/total_predictions*100:.2f}%")

# Loooking where they are different
differences = old_predictions[old_predictions['class'] != new_predictions['class']]
print(f"\nFirst 10 differences:")
print(differences[['ID']].head(10))

# Also checking difference and distribution according to count
print("\nOld model class distribution:")
print(old_predictions['class'].value_counts().sort_index())
print("\nNew model class distribution:")
print(new_predictions['class'].value_counts().sort_index())

Total predictions: 2845
Changed predictions: 2473
Percentage changed: 86.92%

First 10 differences:
    ID
0    1
1    2
2    3
3    4
4    5
5    6
7    8
8    9
9   10
10  11

Old model class distribution:
class
farm           459
forest        1736
grass          123
impervious     418
orchard          7
water          102
Name: count, dtype: int64

New model class distribution:
class
farm            54
forest         128
grass          324
impervious    2248
orchard         14
water           77
Name: count, dtype: int64


In [6]:
# Creating a comparison dataframe
comparison = pd.DataFrame({
    'ID': old_predictions['ID'],
    'old_pred': old_predictions['class'],
    'new_pred': new_predictions['class'],
    'changed': old_predictions['class'] != new_predictions['class']
})

# Looking at sample changes
print("Sample prediction changes:")
changed_samples = comparison[comparison['changed']].head(10)
for _, row in changed_samples.iterrows():
    print(f"ID {row['ID']}: {row['old_pred']} → {row['new_pred']}")

# Checking that new model is confident in which of the classes
print("\nClass changes summary:")
change_summary = comparison[comparison['changed']].groupby(['old_pred', 'new_pred']).size()
print(change_summary)

Sample prediction changes:
ID 1: farm → impervious
ID 2: forest → impervious
ID 3: orchard → impervious
ID 4: farm → grass
ID 5: orchard → impervious
ID 6: farm → grass
ID 8: forest → grass
ID 9: farm → impervious
ID 10: farm → impervious
ID 11: forest → impervious

Class changes summary:
old_pred    new_pred  
farm        forest           8
            grass          121
            impervious     325
            water            4
forest      farm             2
            grass          150
            impervious    1504
            orchard          1
            water           19
grass       forest           1
            impervious     119
impervious  farm            40
            forest          36
            grass           32
            orchard          4
            water           26
orchard     grass            2
            impervious       5
water       farm            11
            forest          23
            grass           16
            impervious      15
     

In [7]:
print("=== MODEL COMPARISON ===")
print(f"Old submission score: 0.53136")
print(f"New model validation accuracy: {model.score(x_scaled, y_new):.5f}")
print(f"Predictions changed: {predictions_changed}/{total_predictions}")

if model.score(X_test_scaled, y_test) > 0.53136:
    print("✅ Validation accuracy improved - good sign!")
else:
    print("⚠️  Validation accuracy not clearly better")

if predictions_changed > total_predictions * 0.1:  # More than 10% changed
    print("✅ Significant prediction changes - could be much better!")
elif predictions_changed > 0:
    print("✅ Some changes - might be slightly better")
else:
    print("❌ No changes - same score expected")

=== MODEL COMPARISON ===
Old submission score: 0.53136
New model validation accuracy: 1.00000
Predictions changed: 2473/2845
✅ Validation accuracy improved - good sign!
✅ Significant prediction changes - could be much better!


In [8]:
data=dataset.copy()
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler,PowerTransformer,MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

# Drop ID column

# Encode class column (if it's categorical)
label_encoder3 = LabelEncoder()
data['class'] = label_encoder3.fit_transform(data['class'])

# Split into features and target
X = data.iloc[:,3:]
y = data['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)
powT=RobustScaler()
X_train_pow=powT.fit_transform(X_train)
X_test_pow=powT.transform(X_test)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_pow)
X_test_scaled = scaler.transform(X_test_pow)

# Fit multinomial logistic regression
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=2000
)
model.fit(X_train_scaled, y_train)
# Predict on test set
y_pred = model.predict(X_test_scaled)

# Classification report with all original class labels
'''print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder3.classes_))),
    target_names=label_encoder3.classes_
))'''
model.score(X_train_scaled,y_train)
model.score(X_test_scaled,y_test)
import matplotlib.pyplot as plt
import seaborn as sns
y_test_decoded=label_encoder3.inverse_transform(y_test)
y_pred_decoded=label_encoder3.inverse_transform(y_pred)
cm=confusion_matrix(y_test_decoded,y_pred_decoded)
plt.figure(figsize=(10, 8))
'''sns.heatmap(cm, 
            annot=True,  # Show numbers in cells
            fmt='d',     # Format as integers
            cmap='Blues',
            xticklabels=label_encoder3.classes_,
            yticklabels=label_encoder3.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()'''
old_pred=pd.read_csv("/kaggle/input/newprediction21/NewPrediction.csv")
old_pred["class"].value_counts()
new_dataset=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
ids=new_dataset["ID"]
x_new=new_dataset.iloc[:,2:]
x_new
x_pow=powT.transform(x_new)
x_scaled=scaler.transform(x_pow)
y_new=model.predict(x_scaled)
y_pred_new=label_encoder3.inverse_transform(y_new)
y_pred_new
dfinal=pd.DataFrame({"ID":ids,"class":y_pred_new})
old_pred["class"].value_counts()
dfinal["class"].value_counts()
#dfinal.to_csv("NewPrediction23.csv",index=False)
#model.score(X_train_scaled,y_train)
model.score(X_test_scaled,y_test)
oc=old_pred["class"].value_counts()
dfi=dfinal["class"].value_counts()
diff=oc-dfi
diff


class
forest        0
farm          0
impervious    0
grass         0
water         0
orchard       0
Name: count, dtype: int64

<Figure size 1000x800 with 0 Axes>

In [9]:
data=dataset.copy()
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

# Drop ID column

# Encode class column (if it's categorical)
label_encoder3 = LabelEncoder()
data['class'] = label_encoder3.fit_transform(data['class'])

# Split into features and target
X = data.iloc[:,3:]
y = data['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Fit multinomial logistic regression
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=2000,
C=1)
model.fit(X_train_scaled, y_train)
# Predict on test set
y_pred = model.predict(X_test_scaled)

# Classification report with all original class labels
'''print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder3.classes_))),
    target_names=label_encoder3.classes_
))'''
model.score(X_train_scaled,y_train)
model.score(X_test_scaled,y_test)
import matplotlib.pyplot as plt
import seaborn as sns
y_test_decoded=label_encoder3.inverse_transform(y_test)
y_pred_decoded=label_encoder3.inverse_transform(y_pred)
cm=confusion_matrix(y_test_decoded,y_pred_decoded)
plt.figure(figsize=(10, 8))
'''sns.heatmap(cm, 
            annot=True,  # Show numbers in cells
            fmt='d',     # Format as integers
            cmap='Blues',
            xticklabels=label_encoder3.classes_,
            yticklabels=label_encoder3.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()'''
old_pred=pd.read_csv("/kaggle/input/newprediction21/NewPrediction.csv")
old_pred["class"].value_counts()
new_dataset=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
ids=new_dataset["ID"]
x_new=new_dataset.iloc[:,2:]
x_new
x_scaled=scaler.transform(x_new)
x_scaled
y_new=model.predict(x_scaled)
y_pred_new=label_encoder3.inverse_transform(y_new)
y_pred_new
dfinal=pd.DataFrame({"ID":ids,"class":y_pred_new})
old_pred["class"].value_counts()
dfinal["class"].value_counts()
dfinal.to_csv("NewPrediction234.csv",index=False)
model.score(X_train_scaled,y_train)
model.score(X_test_scaled,y_test)
dfinal["class"].value_counts()
#old_pred["class"].value_counts()
from sklearn.model_selection import cross_val_score
X_scaled=scaler.transform(X)
scores = cross_val_score(model, X_scaled, y, cv=5)
dfinal["class"].value_counts()


class
forest        1736
farm           459
impervious     418
grass          123
water          102
orchard          7
Name: count, dtype: int64

<Figure size 1000x800 with 0 Axes>

In [10]:
training1=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
Y1=pd.read_csv("/kaggle/working/NewPrediction23.csv")
training1["class"]=Y1["class"]
training12=training1.copy()
training12.iloc[:,2:29]

,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,20150226_N,20150210_N,20150125_N,...,20140610_N,20140525_N,20140509_N,20140423_N,20140407_N,20140322_N,20140218_N,20140202_N,20140117_N,20140101_N
0,7466.4200,413.162,5761.000,5625.45,489.4030,3923.84,3097.110,6766.42000,2829.130,6742.570,...,801.184,927.115,4704.14,6378.42,340.949,2695.5700,527.268,4736.75,601.843,6639.760
1,7235.2600,6037.350,1027.560,6085.14,1618.0500,6668.54,2513.990,1051.69000,7268.220,6908.180,...,5533.470,5103.040,5216.12,4885.27,4366.790,1234.1400,3298.110,6942.68,1070.440,842.101
2,7425.0800,6969.980,1177.940,7408.93,861.0610,7644.43,814.458,1504.29000,7002.630,6086.560,...,1981.390,6204.540,7021.69,5704.41,4897.450,1789.9900,2206.100,6928.93,1036.560,831.441
3,7119.1200,1731.620,6311.930,6441.61,465.9790,7128.42,1649.120,6935.22000,2176.800,690.408,...,959.344,5794.150,1045.57,5572.90,586.287,685.9060,1287.000,6734.72,824.584,6883.610
4,7519.5500,8130.260,1482.540,7879.53,1001.2100,7937.60,4122.530,1094.51000,7951.440,8001.350,...,7636.070,6996.760,7413.43,4596.13,4511.700,1413.5200,3283.940,7937.68,1857.800,1336.920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2840,-1673.7400,-2514.480,-2451.190,-2738.44,64.4464,-2275.03,-2881.100,-4738.97000,-4293.570,-4402.210,...,-2257.890,-2582.420,-1699.10,-2295.30,306.483,209.4030,221.920,-4885.15,-2630.590,356.233
2841,-96.8233,-412.727,-1795.400,-2363.82,-2168.1900,-2162.68,-3155.740,-4416.11000,-5648.660,-5193.280,...,-3991.910,-2614.910,-2109.84,-2687.18,-2853.890,71.3402,-86.414,-5109.01,324.637,-4316.580
2842,-2364.6000,-155.592,-1422.090,-1713.40,465.6220,-2230.40,-3088.730,-5010.32000,-4211.420,-3354.430,...,-2484.500,-1756.080,-3820.43,-1988.23,433.852,291.8000,254.548,-4259.30,412.115,-1170.750
2843,-3004.6300,-1217.120,180.122,-1113.89,438.4180,-2442.51,-3210.560,-3237.74000,-192.857,-2857.540,...,-3291.490,-2018.450,-3472.65,-3109.07,511.792,369.0970,276.948,-5574.51,298.739,460.419


In [11]:
training12=training1.copy()
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix


data=training12.copy()
# Encode class column (if it's categorical)
label_encoder3 = LabelEncoder()
data['class'] = label_encoder3.fit_transform(data['class'])

# Split into features and target
X = data.iloc[:,2:29]
y = data['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.95, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Fit multinomial logistic regression
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=2000,
C=1)
model.fit(X_train_scaled, y_train)
# Predict on test set
y_pred = model.predict(X_test_scaled)

# Classification report with all original class labels
'''print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder3.classes_))),
    target_names=label_encoder3.classes_
))'''
model.score(X_train_scaled,y_train)
model.score(X_test_scaled,y_test)
import matplotlib.pyplot as plt
import seaborn as sns
y_test_decoded=label_encoder3.inverse_transform(y_test)
y_pred_decoded=label_encoder3.inverse_transform(y_pred)
cm=confusion_matrix(y_test_decoded,y_pred_decoded)
plt.figure(figsize=(10, 8))
'''sns.heatmap(cm, 
            annot=True,  # Show numbers in cells
            fmt='d',     # Format as integers
            cmap='Blues',
            xticklabels=label_encoder3.classes_,
            yticklabels=label_encoder3.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()'''
model.score(X_train_scaled,y_train)
model.score(X_test_scaled,y_test)
new_data1=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
x12=new_data1.iloc[:,2:]
x12_scaled=scaler.transform(x12)
y_pred12=model.predict(x12)
y_final=label_encoder3.inverse_transform(y_pred12)
Y1["class"].value_counts()
Y2=pd.read_csv("/kaggle/working/NewPrediction23.csv")
ids=Y1["ID"]
new_data1=pd.DataFrame({"ID":ids,"class":y_final})
nd=new_data1["class"].value_counts()
yy=Y2["class"].value_counts()
diff=nd-yy
diff
#new_data1.to_csv("Prediction4.csv",index=False)
new_data1["class"].value_counts()

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


class
forest        2794
water           28
farm            10
impervious       7
grass            6
Name: count, dtype: int64

<Figure size 1000x800 with 0 Axes>

In [12]:
new_data1=pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
x12=new_data1.iloc[:,2:]
x12_scaled=scaler.transform(x12)
y_pred12=model.predict(x12)
y_final=label_encoder3.inverse_transform(y_pred12)
model.score(x12_scaled,Y1["class"])
Y1["class"].count()

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


2845

In [13]:
def robust_preprocessing(df):
    """Additional techniques for handling noisy data"""
    
    # 1. Median filtering for time series smoothing
    from scipy.signal import medfilt
    
    ndvi_cols = [col for col in df.columns if col.endswith('_N')]
    df_smooth = df.copy()
    
    for idx in df.index:
        ts_data = df.loc[idx, ndvi_cols].values
        # Apply median filter to smooth noise
        if not np.isnan(ts_data).all():
            smoothed = medfilt(ts_data, kernel_size=3)
            df_smooth.loc[idx, ndvi_cols] = smoothed
    
    return df_smooth
#print(robust_preprocessing(dataset))
dataset

,Unnamed: 0,ID,class,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,...,20140610_N,20140525_N,20140509_N,20140423_N,20140407_N,20140322_N,20140218_N,20140202_N,20140117_N,20140101_N
0,0,1,water,637.5950,658.668,-1882.030,-1924.36,997.904,-1739.9900,630.087,...,1537.140,-1043.160,-1942.490,267.138,754.153,764.003,211.328,-2203.020,-1180.190,433.906
1,1,2,water,634.2400,593.705,-1625.790,-1672.32,914.198,-692.3860,707.626,...,1537.140,-933.934,-625.385,120.059,364.858,476.972,220.878,-2250.000,-1360.560,524.075
2,3,4,water,58.0174,-1599.160,-148.529,-1052.63,458.498,-1564.6300,753.058,...,-1025.880,368.622,821.745,-1227.800,304.621,764.003,369.214,-2202.120,804.372,-1343.550
3,4,5,water,72.5180,750.756,380.436,-1256.93,515.805,-1413.1800,-802.942,...,-1813.950,155.624,821.745,-924.073,432.150,282.833,298.320,-2197.360,804.372,-826.727
4,7,8,water,1136.4400,750.756,-148.529,1647.83,1935.800,512.6525,2158.980,...,1535.000,1959.430,-279.317,-384.915,-113.406,1020.720,1660.650,-116.801,-568.050,-1357.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,10537,10538,impervious,1207.7000,984.620,1103.030,1166.25,937.478,1072.7000,823.896,...,1117.740,1176.600,1044.110,1178.140,369.082,465.843,362.882,979.795,1240.080,433.659
7996,10538,10539,impervious,2170.3500,1419.720,1361.000,1478.71,983.911,1262.1100,1422.860,...,984.634,2128.970,1379.660,1178.140,762.633,485.204,446.724,771.747,1589.060,506.936
7997,10541,10542,impervious,1895.6800,1454.740,1103.030,1033.56,1930.380,1057.1500,1471.600,...,888.408,2093.020,1232.110,1190.830,1441.460,1170.880,1095.000,1818.650,2501.720,1247.770
7998,10542,10543,impervious,3465.7400,1283.320,413.412,4391.05,1146.820,4473.0500,1614.750,...,5833.760,4047.320,4515.800,433.177,277.296,744.143,484.877,3759.710,1240.080,388.346


In [14]:
import pandas as pd
import numpy as np
from scipy.signal import medfilt

def robust_preprocessing(df):
    """Additional techniques for handling noisy data"""
    
    # Check if input is DataFrame
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected DataFrame, got {type(df)}. Make sure you pass a DataFrame, not a Series!")
    
    # Find NDVI columns
    ndvi_cols = [col for col in df.columns if col.endswith('_N')]
    
    # Check if NDVI columns exist
    if not ndvi_cols:
        print("Warning: No columns ending with '_N' found!")
        print("Available columns:", list(df.columns))
        return df
    
    print(f"Found {len(ndvi_cols)} NDVI columns")
    df_smooth = df.copy()
    
    # Process each row
    for idx in df.index:
        try:
            # Get NDVI time series for this row
            ts_data = df.loc[idx, ndvi_cols].values
            
            # Check if we have valid data (not all NaN)
            if not np.isnan(ts_data).all():
                # Handle NaN values before median filtering
                if np.isnan(ts_data).any():
                    # Simple interpolation for NaN values
                    ts_data = pd.Series(ts_data).interpolate().values
                
                # Apply median filter only if we have enough valid points
                if len(ts_data) >= 3:
                    smoothed = medfilt(ts_data, kernel_size=3)
                    df_smooth.loc[idx, ndvi_cols] = smoothed
                    
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            continue
    
    return df_smooth
dataset5=robust_preprocessing(dataset.iloc[:,3:])

Found 27 NDVI columns
